In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [167]:
#計測地点のosm_idを指定
start_osm = 315917801
mid_osm = []
end_osm = 315499982
start_mesh = 50325623342
end_mesh = 50325613134

In [168]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [169]:
getdbname = "kddi_18_new_param"
dbname = "kddi_trkn18_monthly_anbun_4a_34"
trknName = "trkn_18_grp"
sttime = "2022-04-01 18:00:00"

In [170]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        #cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        #cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        #cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            #cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            #cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        #cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値26.402186531870697距離0.8590233749368513 km
存在メッシュ：50325623342値14.15距離0.0 km
存在メッシュ：50325623321値18.038998098944298距離0.2726648230002139 km
存在メッシュ：50325623114値23.366239153210785距離0.6461675098581355 km
存在メッシュ：50325623132値21.928021103849645距離0.5453313922032513 km
存在メッシュ：50325623112値24.86503067711915距離0.7512505454326793 km
存在メッシュ：50325623324値15.797936547989405距離0.11553986804341464 km
存在メッシュ：50325623134値20.591083225506985距離0.4515962139681325 km
存在メッシュ：50325623143値19.507502057306667距離0.3756243415432246 km
存在メッシュ：50325623323値16.791432691970773距離0.1851957134200454 km
存在メッシュ：50325613332値27.96490714528253距離0.9685885967790505 km
存在メッシュ：50325613312値31.138728476603482距離1.1911110587401013 km
存在メッシュ：50325613314値29.545408225024474距離1.07940043987984 km
存在メッシュ：50325613134値32.741573033707866距離1.303489444211249 km
存在メッシュ：50325613334値26.402186531870697距離0.8590233749368513 km
存在メッシュ：50325613332値27.96490714528253距離0.9685885967790505 km
3

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値12.922309750608584距離0.8590233749368513 km
存在メッシュ：50325623342値5.779661016949152距離0.0 km
存在メッシュ：50325623321値8.046827556927322距離0.2726648230002139 km
存在メッシュ：50325623114値11.1524456007905距離0.6461675098581355 km
50325623132 is kddi none!
存在メッシュ：50325623112値12.026195061951297距離0.7512505454326793 km
50325623324 is kddi none!
存在メッシュ：50325623134値9.534614863300535距離0.4515962139681325 km
50325623143 is kddi none!
存在メッシュ：50325623323値7.319535211916295距離0.1851957134200454 km
存在メッシュ：50325613332値13.833327908993189距離0.9685885967790505 km
存在メッシュ：50325613312値15.683568341919381距離1.1911110587401013 km
存在メッシュ：50325613314値14.75471153357326距離1.07940043987984 km
存在メッシュ：50325613134値16.617977528089888距離1.303489444211249 km
存在メッシュ：50325613334値12.922309750608584距離0.8590233749368513 km
存在メッシュ：50325613332値13.833327908993189距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値11.879304781570077距離0.8

存在メッシュ：50325613314値6.946379385906763距離1.07940043987984 km
存在メッシュ：50325613134値7.876712328767123距離1.303489444211249 km
存在メッシュ：50325613334値6.0314570139722345距離0.8590233749368513 km
存在メッシュ：50325613332値6.486330449532921距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値5.806382905007997距離0.8590233749368513 km
50325623342 is kddi none!
存在メッシュ：50325623321値3.415038479985684距離0.2726648230002139 km
存在メッシュ：50325623114値4.938293432094002距離0.6461675098581355 km
50325623132 is kddi none!
存在メッシュ：50325623112値5.366853279060745距離0.7512505454326793 km
存在メッシュ：50325623324値2.7742362202986337距離0.11553986804341464 km
存在メッシュ：50325623134値4.144773866021049距離0.4515962139681325 km
存在メッシュ：50325623143値3.834937993444841距離0.3756243415432246 km
50325623323 is kddi none!
存在メッシュ：50325613332値6.253222439561231距離0.9685885967790505 km
存在メッシュ：50325613312値7.160735167217842距離1.1911110587401013 km
存在メッシュ：50325613314値6.705146066069805距離1.07940043987984 km
存在メッシュ：50325613134値7.6190476

存在メッシュ：50325613334値21.951614947275758距離0.8590233749368513 km
存在メッシュ：50325613332値23.297434009040778距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値21.666153279176022距離0.8590233749368513 km
存在メッシュ：50325623342値10.133333333333333距離0.0 km
存在メッシュ：50325623321値13.793995430517475距離0.2726648230002139 km
50325623114 is kddi none!
存在メッシュ：50325623132値17.454680971350932距離0.5453313922032513 km
50325623112 is kddi none!
存在メッシュ：50325623324値11.684514050399478距離0.11553986804341464 km
存在メッシュ：50325623134値16.196239158569565距離0.4515962139681325 km
50325623143 is kddi none!
存在メッシュ：50325623323値12.619678734321141距離0.1851957134200454 km
存在メッシュ：50325613332値23.137121372361246距離0.9685885967790505 km
存在メッシュ：50325613312値26.124597106523993距離1.1911110587401013 km
50325613314 is kddi none!
50325613134 is kddi none!
存在メッシュ：50325613334値21.666153279176022距離0.8590233749368513 km
存在メッシュ：50325613332値23.137121372361246距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値8.511951857905768距離0.8590233749368513 km
50325623342 is kddi none!
存在メッシュ：50325623321値5.161959410770578距離0.2726648230002139 km
存在メッシュ：50325623114値7.295860546562228距離0.6461675098581355 km
存在メッシュ：50325623132値6.719762131306573距離0.5453313922032513 km
50325623112 is kddi none!
存在メッシュ：50325623324値4.2642707685170675距離0.11553986804341464 km
存在メッシュ：50325623134値6.184232909329008距離0.4515962139681325 km
存在メッシュ：50325623143値5.750189271732908距離0.3756243415432246 km
50325623323 is kddi none!
50325613332 is kddi none!
存在メッシュ：50325613312値10.409240178939285距離1.1911110587401013 km
50325613314 is kddi none!
存在メッシュ：50325613134値11.051282051282051距離1.303489444211249 km
存在メッシュ：50325613334値8.511951857905768距離0.8590233749368513 km
50325613332 is kddi none!
33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値8.207020512639126距離0.8590233749368513 km
50325623342 is kddi none!
存在メッシュ：50325623321値4.710866345201885距離0.272

...

存在メッシュ：50325623324値10.662043185225485距離0.11553986804341464 km
存在メッシュ：50325623134値14.284313914927754距離0.4515962139681325 km
50325623143 is kddi none!
50325623323 is kddi none!
存在メッシュ：50325613332値19.856850778000506距離0.9685885967790505 km
存在メッシュ：50325613312値22.25536701028545距離1.1911110587401013 km
存在メッシュ：50325613314値21.051265049999607距離1.07940043987984 km
存在メッシュ：50325613134値23.466666666666665距離1.303489444211249 km
50325613334 is kddi none!
存在メッシュ：50325613332値19.856850778000506距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
50325613334 is kddi none!
存在メッシュ：50325623342値7.7距離0.0 km
存在メッシュ：50325623321値11.06083643970049距離0.2726648230002139 km
存在メッシュ：50325623114値15.664589232253268距離0.6461675098581355 km
存在メッシュ：50325623132値14.421694402903775距離0.5453313922032513 km
存在メッシュ：50325623112値16.95983110198647距離0.7512505454326793 km
存在メッシュ：50325623324値9.124131629763585距離0.11553986804341464 km
存在メッシュ：50325623134値13.266324967169265距離0.4515962139681325 km
存在メッシュ：50325623143値12

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値8.474174816654084距離0.8590233749368513 km
50325623342 is kddi none!
存在メッシュ：50325623321値5.340788727163906距離0.2726648230002139 km
50325623114 is kddi none!
存在メッシュ：50325623132値6.797865855463611距離0.5453313922032513 km
存在メッシュ：50325623112値7.8982577712250945距離0.7512505454326793 km
50325623324 is kddi none!
存在メッシュ：50325623134値6.2969632889235距離0.4515962139681325 km
存在メッシュ：50325623143値5.890984374720757距離0.3756243415432246 km
50325623323 is kddi none!
50325613332 is kddi none!
存在メッシュ：50325613312値10.248786827484892距離1.1911110587401013 km
存在メッシュ：50325613314値9.651827001404097距離1.07940043987984 km
存在メッシュ：50325613134値10.849315068493151距離1.303489444211249 km
存在メッシュ：50325613334値8.474174816654084距離0.8590233749368513 km
50325613332 is kddi none!
33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値7.494062727687587距離0.8590233749368513 km
50325623342 is kddi none!
存在メッシュ：50325623321値4.712206301015584距離0.27266482

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値24.07190036794408距離0.8590233749368513 km
存在メッシュ：50325623342値12.05距離0.0 km
存在メッシュ：50325623321値15.8659023742429距離0.2726648230002139 km
存在メッシュ：50325623114値21.093015185806888距離0.6461675098581355 km
存在メッシュ：50325623132値19.681829186326468距離0.5453313922032513 km
存在メッシュ：50325623112値22.563636150144625距離0.7512505454326793 km
存在メッシュ：50325623324値13.666962730781856距離0.11553986804341464 km
存在メッシュ：50325623134値18.370019744823296距離0.4515962139681325 km
存在メッシュ：50325623143値17.306805043447397距離0.3756243415432246 km
存在メッシュ：50325623323値14.64178560242296距離0.1851957134200454 km
存在メッシュ：50325613332値25.605248841581876距離0.9685885967790505 km
存在メッシュ：50325613312値28.719416564342662距離1.1911110587401013 km
存在メッシュ：50325613314値27.15604358851629距離1.07940043987984 km
存在メッシュ：50325613134値30.292134831460675距離1.303489444211249 km
存在メッシュ：50325613334値24.07190036794408距離0.8590233749368513 km
存在メッシュ：50325613332値25.605248841581876距離0.9685885967790505 km
33.7

存在メッシュ：50325613314値12.150707856676982距離1.07940043987984 km
存在メッシュ：50325613134値13.662650602409638距離1.303489444211249 km
存在メッシュ：50325613334値10.66380980177004距離0.8590233749368513 km
50325613332 is kddi none!
33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値9.5849965114857距離0.8590233749368513 km
50325623342 is kddi none!
50325623321 is kddi none!
存在メッシュ：50325623114値8.265183376745988距離0.6461675098581355 km
存在メッシュ：50325623132値7.639948866586453距離0.5453313922032513 km
存在メッシュ：50325623112値8.916750908316306距離0.7512505454326793 km
50325623324 is kddi none!
存在メッシュ：50325623134値7.058743742752407距離0.4515962139681325 km
存在メッシュ：50325623143値6.587680025245337距離0.3756243415432246 km
50325623323 is kddi none!
50325613332 is kddi none!
存在メッシュ：50325613312値11.644106730538077距離1.1911110587401013 km
存在メッシュ：50325613314値10.951444858114918距離1.07940043987984 km
存在メッシュ：50325613134値12.340909090909092距離1.303489444211249 km
存在メッシュ：50325613334値9.5849965114857距離0.8590233749368513 km
50325613332 i

存在メッシュ：50325613314値6.645873382740058距離1.07940043987984 km
存在メッシュ：50325613134値7.6距離1.303489444211249 km
存在メッシュ：50325613334値5.707551468538682距離0.8590233749368513 km
存在メッシュ：50325613332値6.174058492377424距離0.9685885967790505 km
33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値5.321320160376136距離0.8590233749368513 km
存在メッシュ：50325623342値2.2916666666666665距離0.0 km
存在メッシュ：50325623321値3.2533167890221835距離0.2726648230002139 km
50325623114 is kddi none!
存在メッシュ：50325623132値4.214973069987163距離0.5453313922032513 km
存在メッシュ：50325623112値4.941220690488867距離0.7512505454326793 km
存在メッシュ：50325623324値2.6991593788483286距離0.11553986804341464 km
存在メッシュ：50325623134値3.8843824032961844距離0.4515962139681325 km
存在メッシュ：50325623143値3.6164403944623826距離0.3756243415432246 km
50325623323 is kddi none!
存在メッシュ：50325613332値5.7077411435859045距離0.9685885967790505 km
存在メッシュ：50325613312値6.4925462769254825距離1.1911110587401013 km
存在メッシュ：50325613314値6.098558783000399距離1.07940043987984 km
存在メッシュ：50325613134

In [171]:
connection.close()

In [172]:
#CreateGeojsonFile